## 기존 내용 2020.04.26

In [1]:
import numpy as np 
import pandas as pd
import os
import io
import glob
import re
import sys
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import plotly.express as px
import plotly.graph_objects as go

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset/", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

['SeoulFloating',
 'TimeAge',
 'SearchTrend',
 'TimeProvince',
 'Weather',
 'SFedit',
 'PatientRoute',
 'PatientInfo',
 'Region',
 'TimeGender',
 'Policy',
 'Case',
 'Time']

20대 확진자가 진짜진짜 많다는 소문을 듣고 확인하러 왔습니다

In [3]:
patientcount = PatientInfo.groupby('age').count().patient_id

In [4]:
fig = patientcount.iplot(asFigure=True, kind='bar')
fig.show()

이제부터 20대 확진자가 왜 많은지 알아보겠습니다. 내가 20대라 무섭거든요. 혹시 20대 전체 인구가 많은 건 아닐까요? 전체 인구 확인을 위해 행정안전부에서 전체 연령별 인구 데이터를 가져왔습니다.

In [5]:
population = pd.read_csv('custom_data/korea_2020_pop.csv',encoding='CP949')
population.columns = ['region','total','total2','0s','10s','20s','30s','40s','50s','60s','70s','80s','90s','100s']
totalpop = population.iloc[0,[3,4,5,6,7,8,9,10,11,12,13]]

In [6]:
fig = totalpop.iplot(asFigure=True, kind='bar')
fig.show()

50대가 가장 많음을 확인할 수 있었습니다. 20대가 많았으면 바로 접고 맥주나 마셨을텐데 아쉬워요. 20대가 많이 돌아다녀서 코로나에 많이 걸린걸까요? 유동인구 데이터를 확인해봅시다.

In [7]:
birthsum = SFedit.groupby('birth_year').sum()

In [8]:
fig = birthsum.iplot(asFigure=True, kind='bar')
fig.show()

이럴수가! 30대랑 40대가 더 많이 돌아다녔네요. 도대체 20대는 코로나를 어디서 걸려버린걸까요. 혹시 20대가 사람 많은 곳에 가서 앉아 있는 걸 좋아해서 그런 건 아닐까요?

In [9]:
agecontact = PatientInfo.groupby('age').mean().contact_number

In [10]:
fig = agecontact.iplot(asFigure=True, kind='bar')
fig.show()

확진자의 평균 접촉자 수를 나타낸 그래프입니다. 10대가 월등히 높은 이유는 outlier 한명이 1000명 넘는 접촉자를 가지고 있고, 10대 확진자 수가 상대적으로 적어서고요. 10대를 빼고 봐도 20대는 접촉자 수가 그렇게 많진 않네요. 오히려 60대의 접촉자 수가 많아요! 도대체! 20대는 코로나를 어디서 걸려온 걸까요! 혹시 대구에 20대가 많은 것은 아니었을까요?

In [11]:
twtregion = population.loc[:,['region','20s']]
twtregion = twtregion.drop(0)
twtregion = twtregion.set_index('region')

In [12]:
fig = twtregion.iplot(asFigure=True,kind='bar')
fig.show()

시도별 인구 현황을 그려봤는데, 서울이랑 경기도에 20대가 제일 많아요. 아래에서 보듯이 확진자 수는 대구에 제일 많은데 말이에요..

In [13]:
TP = TimeProvince[TimeProvince.date=='2020-04-20'].drop(['released','deceased','time','date'],axis=1)
regionconf = TP.set_index('province')

In [14]:
fig = regionconf.iplot(asFigure=True,kind='bar')
fig.show()

대구에 저렇게나 확진자가 많으니까, 아마 20대 확진자도 대구에 제일 많겠죠? 지역별 확진자 수를 봅시다.

In [15]:
twtinfo = PatientInfo[PatientInfo.age == '20s']
twtinforeg = twtinfo.groupby('province').count().patient_id

In [16]:
fig = twtinforeg.iplot(asFigure=True,kind='bar')
fig.show()

아니 이럴수가. 아니었네요. 대구보다 경상북도에 20대 확진자가 훨씬 많아요! 그리고 그 다음으로 서울이랑 경기도에 많구요. 서울이랑 경기도에 20대 확진자가 많은 건 아무래도 그 두 지역에 20대가 많아서겠고, 도대체 경상북도에 20대 코로나 환자가 왜 그렇게 많아진걸까요?! 우리는 이 미스테리를 풀어보고자 합니다... *투비컨티뉴드*

## 내용추가 2020.04.28

In [44]:
simpcase = Case.loc[:,['infection_case','confirmed']]

In [47]:
simpcase = simpcase.groupby('infection_case').sum()

In [48]:
fig = simpcase.iplot(asFigure=True,kind='bar')
fig.show()

In [23]:
gsb = Case[Case.province == 'Gyeongsangbuk-do']
gsb = gsb.loc[:,['infection_case','confirmed']]

In [26]:
gsb = gsb.set_index('infection_case')

In [27]:
fig = gsb.iplot(asFigure=True, kind='bar')
fig.show()

In [31]:
twtgsb = twtinfo[twtinfo.province == 'Gyeongsangbuk-do']

In [34]:
twtgsbinf = twtgsb.groupby('infection_case').count().patient_id
twtgsbinf = twtgsbinf.reset_index()

In [55]:
fig = px.pie(twtgsbinf,values='patient_id',names = 'infection_case', title='Infection Case of Twenties in Gyeongsangbuk-do')
fig.show()

The value 'etc' includes individual cases, cases where relevance classification is ongoing after investigation, and cases under investigation. 즉 아직 분류가 안된 사람들이구나.

경상북도 전체 인원을 놓고 보면, 신천지에 의한 감염이 가장 많았는데 20대만 놓고 보면 etc 가 가장 높았고 이건 더 분석할 수 없으니 그 다음으로는 contact with patient가 많았다. (왜 유난히 etc 가 많은 건지....)